### Imports

In [16]:
import requests
from requests.exceptions import MissingSchema
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from random import randint
from time import sleep
from tqdm import tqdm
import recomendaciones_escritoras.data.load as load

### Load data

In [17]:
goodreads_escritoras = load.goodreads_escritoras
data_processed = load.data_processed

### Read data

In [18]:
# Read csv file as dataframe
df = pd.read_csv(goodreads_escritoras, converters={'id': str})

# Convert columns to lists
nombres = df['nombre'].to_list()
urls = df['url'].to_list()

### Scrape data

In [19]:
# Create an empty list
dfs = []
# Iterate trhough
for i in tqdm(range(len(urls))):

    try:
        # Make http request
        response = requests.get(urls[i])

        # Return content of the response
        html = response.text
        # Parse html
        soup = bs(html, 'html.parser')

        # Look for <a> tags inside <tr> tag
        results = [tr.find('a', attrs={'class':'bookTitle'}) for tr in soup.find_all('tr')]

        # Extract 'href' from <a> tag and build a link
        book_url = ['https://www.goodreads.com' + i.get("href") for i in results]

        # Create dataframe from list
        df = pd.DataFrame(book_url, columns=['book_url'])

        # Create new column
        df['nombre'] = nombres[i]

        # Append dataframes to list
        dfs.append(df)

        # Sleep a random number of seconds (between 1 and 5)
        sleep(randint(1,5))
    
    except MissingSchema:
        continue


 12%|█▏        | 133/1116 [09:36<1:20:53,  4.94s/it]

In [ ]:
# Join lists on dataframes
df_books = pd.concat(dfs)

# Show dataframe
df_books

### Export data

In [ ]:
# Save dataframe as csv file
df_books.to_csv(f'{data_processed}/goodreads_libros.csv', index=False)